In [9]:
from binance.client import Client
from binance.enums import *
from time import time
import pickle as pickle
from datetime import datetime
import pandas as pd
from pathlib import Path
from datetime import datetime
import json
import math
import ast
from time import time
import pickle


from typing import List, Dict


downloads_path = str(Path.home() / "Downloads")
documents_path = str(Path.home() / "Documents")
tst_fn = f'{downloads_path}\\hd.csv'
hd_dl_fn = f'{downloads_path}\\hd_dl.csv'
qualifying_trades_fn = f'{downloads_path}\\qualifying_trades.csv'


def get_client():
    fn = f'{documents_path}\\key\\binance-key.pickle'
    with open(fn, 'rb') as handle:
        k = pickle.load(handle)
    return Client(k['API_KEY'], k['API_SECRET'])

def get_config(file_path):
    with open(file_path, 'r') as file:
        config = json.load(file)
    return config


def get_chunks(lst, chunk_size):
    r=[]
    tot = len(lst)
    for idx, val in enumerate(lst):
        end_idx = idx+chunk_size
        if end_idx <= tot:
            r.append(lst[idx:end_idx])
    return r

def get_pc(hd_df):
    close_price = list(hd_df['close'])
    ts = list(hd_df['time'])
    pc = []
    for idx, val in enumerate(close_price):
        if idx==0:
            pc.append({'time': ts[0], 'pc': 0})
        else:
            previous_price = close_price[idx-1]
            current_price = val
            pc_val = (round((current_price*100/previous_price)-100, 4))
            pc.append({'time': ts[idx], 'pc': pc_val})
    pc_df = pd.DataFrame(pc)
    return pc_df

def get_trades(hd_df, chunk_size):
    close_price = list(hd_df['close'])
    high_price = list(hd_df['high'])
    low_price = list(hd_df['low'])
    longs=[]
    shorts=[]
    long_qualified = []
    short_qualified = []
    tot = len(close_price)
    for idx in range(len(close_price)):
        if (idx + chunk_size) <= tot:
            cp = close_price[idx]
            max_price = max(high_price[idx:idx + chunk_size])
            min_price = min(low_price[idx:idx + chunk_size])
            long_roi = round((max_price*100/cp)-100, 4)
            short_roi = round((cp*100/min_price)-100, 4)
            longs.append({'long': long_roi})
            long_qualified.append(long_roi >= roi_threshold)
            shorts.append({'short': short_roi})
            short_qualified.append(short_roi >= roi_threshold)
        else:
            long_roi = 0.0
            short_roi = 0.0
            longs.append({'long': long_roi})
            long_qualified.append(False)
            shorts.append({'short': short_roi})
            short_qualified.append(False)
    long_trades_df = pd.DataFrame(longs)
    short_trades_df = pd.DataFrame(shorts)
    long_qualified_df = pd.DataFrame(long_qualified)
    short_qualified_df = pd.DataFrame(short_qualified)
    return long_trades_df, short_trades_df, long_qualified, short_qualified

def download_hd(start_timestamp, end_timestamp, candlestick): 
    client = get_client()
    data = []
    tot = (end_timestamp - start_timestamp)/(900*500)
    cntr = 0
    for current_sts in range(start_timestamp, end_timestamp+1, 900*500):
        next_ets = current_sts + 900*500 if (current_sts + 900*500) < end_timestamp else end_timestamp
        print(current_sts, next_ets, f'100% completed') if next_ets == end_timestamp else print(current_sts, next_ets, f'{round(cntr*100/tot, 1)}% completed')
        cntr += 1
        
        klines = client.futures_historical_klines('BTCUSDT', candlestick, current_sts*1000, next_ets*1000, limit=500)
        
        for kline in klines:
            timestamp = kline[0]/1000
            open_price = float(kline[1])
            high_price = float(kline[2])
            low_price = float(kline[3])
            close_price = float(kline[4])
            volume = float(kline[5])

            data.append([timestamp, open_price, high_price, low_price, close_price, volume])

    df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    df.to_csv(hd_dl_fn, index=False)
    print('Historical Data Exported!\t', hd_dl_fn)
    return df

def read_local_hd(fn):
    return pd.read_csv(fn)

def get_qualifying_trades(df):
    close_pc_df = get_pc(df)
    long_trades_df, short_trades_df, long_qualified, short_qualified = get_trades(df, chunk_size)
    df['close_pc'] = close_pc_df['pc']
    df['long'] = long_trades_df
    df['short'] = short_trades_df
    df['long_qualified'] = long_qualified
    df['short_qualified'] = short_qualified
    df.to_csv(qualifying_trades_fn, index=False)
    print('Qualifying Trades Data Exported!', qualifying_trades_fn)
    return df

def get_chunks(qt_df):
    lst = list(qt_df['close_pc'])
    r=[]
    tot = len(lst)
    for idx, val in enumerate(lst):
        if idx < chunk_size:
            r.append([0]*chunk_size)
        else:
            r.append(lst[idx-chunk_size:idx])
    qt_df['ref_chunks'] = r
    qt_df.to_csv(f'{downloads_path}/ref_chunks.csv', index=False)
    print('Ref Chunks Exported!', f'{downloads_path}/ref_chunks.csv')
    return qt_df

def get_tst_chunks(ref_chunks_df):
    long_qualified = list(ref_chunks_df['long_qualified'])
    short_qualified = list(ref_chunks_df['short_qualified'])
    ref_chunks_lst = list(ref_chunks_df['ref_chunks'])
    r=[]
    tst_lst = []
    chunk_diff_lst = []
    for idx in range(len(long_qualified)):
        if (idx < chunk_size) or ((long_qualified[idx] == False) and (short_qualified[idx] == False)) :
            r.append(None)
            chunk_diff_lst.append(None)
        else:
            if (long_qualified[idx] == True) or (short_qualified[idx] == True):
                r.append(ref_chunks_lst[idx])
                tst_lst.append(ref_chunks_lst[idx])

    ref_chunks_df['tst_chunks'] = r
    ref_chunks_df['sm'] = chunk_diff_lst
    ref_chunks_df.to_csv(f'{downloads_path}/tst_chunks.csv', index=False)
    print('TST Chunks Exported!', f'{downloads_path}/tst_chunks.csv')
    return ref_chunks_df, tst_lst



def sort_dicts_by_value(dicts: List[Dict], key: str) -> List[Dict]:
    """
    Sorts a list of dictionaries by the specified key value.

    :param dicts: List of dictionaries to be sorted
    :param key: The key by which the dictionaries should be sorted
    :return: A list of dictionaries sorted by the specified key
    """
    try:
        return sorted(dicts, key=lambda x: x[key])
    except KeyError:
        raise KeyError(f"One or more dictionaries do not have the key '{key}'")
    except TypeError:
        raise TypeError(f"The value for key '{key}' is not comparable")

def all_zeros(lst):
    return all(x == 0 for x in lst)

def format_elapsed_time(seconds):
    days = seconds // (24 * 3600)
    seconds %= 24 * 3600
    hours = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    return f"{days:03}d:{hours:02}h:{minutes:02}m:{seconds:02}s"

# math.isnan(tst_chunks_lst[110])
def chunk_diff(ref_lst, tst_lst):
    cntr = 0
    chunk_size = len(tst_lst[0])
    r = []
    tot = len(tst_lst)
    start_ts = int(time())
    for tst_lst_idx,tst_val in enumerate(tst_lst):
        tst_lst_idx += 1
        pattern_occurrence = 0
        long_rois_lst = []
        short_rois_lst = []
        avg_long_roi = 0
        avg_short_roi = 0
        avg_sm = 0
        sm_lst = []
        if(tst_lst_idx%10 == 0) and tst_lst_idx != 0:
            completed = round(tst_lst_idx*100/tot, 1)
            remaining = round(100-completed)
            cur_ts = time()
            ts_diff = int(cur_ts - start_ts)
            estimated_tm_to_complete = round(int(ts_diff*(tot-tst_lst_idx)/tst_lst_idx), 1)
            print(f"\rElapsed Time: {format_elapsed_time(ts_diff)} \t Completed: {completed}% \tRemaining: {remaining}% \tETA: {format_elapsed_time(estimated_tm_to_complete)}", end="")
        for ref_idx,ref_val in enumerate(ref_lst):
            diff = []
            if(all_zeros(ref_val)) or (all_zeros(tst_val) or (tst_val == ref_val)):
                continue
            else:
                for k in range(chunk_size):
                    diff.append(abs(tst_val[k] - ref_val[k]))
                cntr += 1
                sm = round(sum(diff), 2)

                if (sm <= sm_threshold) and (long_qualified_lst[ref_idx]):
                    
                    sm_lst.append(sm)
                    long_rois_lst.append(long[ref_idx])
                    short_rois_lst.append(short[ref_idx])
                    pattern_occurrence += 1
                    # sm_lst.append({'sm': sm, 'pattern_id': pattern_id, 'long_roi': long_roi, 'short_roi': short_roi, 'ref': ref_val, 'tst': tst_val})
        if pattern_occurrence > 0:
            pattern_id = f'{ts[tst_lst_idx]}_{candlestick}_{chunk_size}_{roi_threshold}_{sm_threshold}'
            avg_sm = round(sum(sm_lst)/len(sm_lst), 2)
            avg_long_roi = round(sum(long_rois_lst)/len(long_rois_lst), 2)
            avg_short_roi = round(sum(short_rois_lst)/len(short_rois_lst), 2)
            r.append({'avg_sm': avg_sm, 'pattern_occurrence': pattern_occurrence, 'pattern_id': pattern_id, 'avg_long_roi': avg_long_roi, 'avg_short_roi': avg_short_roi, 'ref': ref_val, 'tst_chunk': tst_val})
    return r

def str_to_lst(val):
    try:
        return ast.literal_eval(val)
    except ValueError as e:
        return math.isnan(val)

def save_dict_to_pickle(dictionary, filename):
    with open(filename, 'wb') as file:
        pickle.dump(dictionary, file)
        print(f'Saved {filename}')

def load_dict_from_pickle(filename):
    with open(filename, 'rb') as file:
        dictionary = pickle.load(file)
    print(f'Reading {filename}')
    return dictionary




config = get_config('config.json')
candlestick = config['candlestick']
chunk_size = config['chunk_size']
roi_threshold = config['roi_threshold']
sm_threshold = config['sm_threshold']

start_timestamp = 1609459200 #  January 1, 2021 12:00:00 AM
end_timestamp = 1719721304

# ref_chunks = get_chunks(pc_ref)
# tst_chunks = get_chunks(pc_tst)
# hd = read_local_hd(hd_dl_fn)
# qt_df = get_qualifying_trades(hd)

# ref_chunks_df = get_chunks(qt_df)
# tst_chunks_df, tst_lst = get_tst_chunks(ref_chunks_df)

print('Reading tst_chunks.csv')
tst_chunks_df = pd.read_csv(f'{downloads_path}/tst_chunks.csv')
close_price_lst = list(tst_chunks_df['close'])
high_price_lst = list(tst_chunks_df['high'])
ref_chunks_lst_str = list(tst_chunks_df['ref_chunks'])
tst_chunks_lst_str = list(tst_chunks_df['tst_chunks'])
long_qualified_lst = list(tst_chunks_df['long_qualified'])
ts = list(tst_chunks_df['time'])
ts = [int(i) for i in ts]
long = list(tst_chunks_df['long'])
short = list(tst_chunks_df['short'])

# -------------------------------------------------------------------------------------------



# for idx in range(len(long_qualified_lst)):

tst_val_lst = []
for i in tst_chunks_lst_str:
    v = str_to_lst(i)
    if type(v) == type([1]):
        tst_val_lst.append(v)

tst_val_lst = tst_val_lst[:50]
ref_chunks_lst = [str_to_lst(i) for i in ref_chunks_lst_str]

print('Calculating chunk_diff')
chunk_diff_lst = chunk_diff(ref_chunks_lst, tst_val_lst)
# print('Sorting by avg_sm...')
# sorted_by_avg_sm = sort_dicts_by_value(chunk_diff_lst, 'avg_sm')
print('\nSorting by pattern_occurrence...')
sorted_by_pattern_occurrence = sort_dicts_by_value(chunk_diff_lst, 'pattern_occurrence')
sorted_by_pattern_occurrence = sorted_by_pattern_occurrence[::-1]
print('Sorting completed!')
dict_fn = f'{downloads_path}/sorted_by_pattern_occurrence.pkl'
save_dict_to_pickle(sorted_by_pattern_occurrence, dict_fn)

Reading tst_chunks.csv
Calculating chunk_diff
Elapsed Time: 000d:00h:00m:12s 	 Completed: 100.0% 	Remaining: 0% 	ETA: 000d:00h:00m:00s
Sorting by pattern_occurrence...
Sorting completed!
Saved C:\Users\henok\Downloads/sorted_by_pattern_occurrence.pkl


In [1]:
import pickle
from pathlib import Path


downloads_path = str(Path.home() / "Downloads")


def load_dict_from_pickle(filename):
    with open(filename, 'rb') as file:
        dictionary = pickle.load(file)
    return dictionary

dict_fn = f'{downloads_path}/sorted_by_pattern_occurrence.pkl'
sorted_by_pattern_occurrence = load_dict_from_pickle(dict_fn)


In [2]:
len(sorted_by_pattern_occurrence)

23688

In [4]:
import pandas as pd
sorted_by_pattern_occurrence_df = pd.DataFrame(sorted_by_pattern_occurrence)
sorted_by_pattern_occurrence_df.to_csv(f'{downloads_path}/ana.csv', index=False)

In [5]:
sorted_by_pattern_occurrence_df

,avg_sm,pattern_occurrence,pattern_id,avg_long_roi,avg_short_roi,ref,tst_chunk
0,0.80,1557,1641793500_15m_10_1_1,1.67,0.33,"[-0.0075, -0.0289, -0.0738, -0.0611, -0.0449, ...","[-0.0193, 0.0235, 0.0178, 0.0439, 0.0073, -0.0..."
1,0.80,1550,1641505500_15m_10_1_1,1.69,0.33,"[-0.0075, -0.0289, -0.0738, -0.0611, -0.0449, ...","[0.0383, 0.0984, -0.0021, -0.016, 0.0078, -0.0..."
2,0.80,1535,1643012100_15m_10_1_1,1.71,0.34,"[-0.0075, -0.0289, -0.0738, -0.0611, -0.0449, ...","[0.0206, 0.0353, -0.0441, -0.0583, -0.0029, 0...."
3,0.81,1529,1634284800_15m_10_1_1,1.68,0.33,"[-0.0075, -0.0289, -0.0738, -0.0611, -0.0449, ...","[0.0506, 0.0345, 0.0247, 0.0596, 0.0503, -0.02..."
4,0.80,1524,1643013000_15m_10_1_1,1.70,0.34,"[-0.0075, -0.0289, -0.0738, -0.0611, -0.0449, ...","[0.0353, -0.0441, -0.0583, -0.0029, 0.0648, 0...."
...,...,...,...,...,...,...,...
23683,0.99,1,1609497000_15m_10_1_1,1.12,0.19,"[-0.0075, -0.0289, -0.0738, -0.0611, -0.0449, ...","[-0.5752, 0.2088, 0.1272, 0.1737, 0.0688, 0.01..."
23684,0.81,1,1609477200_15m_10_1_1,1.66,0.68,"[-0.0075, -0.0289, -0.0738, -0.0611, -0.0449, ...","[0.4276, 0.1731, 0.1696, -0.0697, 0.2628, -0.1..."
23685,0.90,1,1609470900_15m_10_1_1,1.68,0.28,"[-0.0075, -0.0289, -0.0738, -0.0611, -0.0449, ...","[0.0173, 0.0729, -0.3921, 0.331, -0.0669, -0.8..."
23686,0.84,1,1609468200_15m_10_1_1,2.99,0.33,"[-0.0075, -0.0289, -0.0738, -0.0611, -0.0449, ...","[-0.5416, 0.1393, -0.279, 0.1289, 0.0536, 0.01..."


In [ ]:
for i in sorted_by_pattern_occurrence:
    print(i)

In [ ]:
sorted_by_pattern_occurrence = sort_dicts_by_value(chunk_diff_lst, 'pattern_occurrence')
sorted_by_pattern_occurrence = sorted_by_pattern_occurrence[::-1]
for i in sorted_by_pattern_occurrence:
    print(i)

In [ ]:
import matplotlib.pyplot as plt

def plot_lists(tst_lsts, ref_lst, tst_lst, label1='List 1', label2='List 2', title='Line Graph', xlabel='X-axis', ylabel='Y-axis'):
    plt.figure(figsize=(10, 6))
    plt.plot(ref_lst, label=label1, marker='o')
    plt.plot(tst_lst, label=label2, marker='x')
    for i in tst_lsts:
        plt.plot(i, label=label2, marker='x')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage
ref_lst = [0.1393, -0.279, 0.1289, 0.0536, 0.0173, 0.0729, -0.3921, 0.331, -0.0669, -0.8499]
tst_lst = [0.1627, -0.3479, -0.0267, -0.1921, 0.0222, 0.1822, 0.036, 0.2573, -0.1122, -0.7797]
tst_lsts = [[0.1393, -0.279, 0.1289, 0.0536, 0.0173, 0.0729, -0.3921, 0.331, -0.0669, -0.8499], [0.1339, -0.2712, 0.1238, 0.11, -0.0212, 0.1032, -0.4732, -0.1148, 0.278, -0.763], [-0.2349, -0.0012, 0.107, 0.0887, -0.0334, 0.087, -0.3967, 0.1344, -0.1688, -0.8242], [0.0698, -0.2854, -0.074, -0.0168, -0.0024, 0.1081, -0.438, -0.0103, 0.042, -0.6446], [0.2045, -0.2967, 0.0766, 0.0257, 0.133, -0.0037, -0.3916, -0.1858, -0.2335, -0.7369], [0.1061, -0.3776, 0.0746, 0.0418, 0.0291, 0.1158, -0.2508, 0.1434, -0.231, -0.4328], [0.1317, -0.0052, 0.1327, -0.0427, 0.0961, 0.2314, -0.1699, 0.3709, -0.1205, -0.6201], [0.0528, -0.1169, -0.1639, 0.0529, 0.0199, 0.093, -0.4968, 0.0811, -0.0742, -1.1212], [0.1627, -0.3479, -0.0267, -0.1921, 0.0222, 0.1822, 0.036, 0.2573, -0.1122, -0.7797], [0.0352, -0.2445, 0.2051, 0.0843, -0.0346, -0.0835, -0.1908, 0.1057, -0.0711, -0.4893], [-0.1001, -0.219, 0.3287, 0.1449, 0.0267, 0.0495, -0.0306, 0.1839, -0.0741, -0.9654], [0.0535, -0.1236, 0.1096, 0.1261, -0.1683, 0.064, -0.3994, 0.2862, 0.0097, -0.2452], [0.1355, -0.1066, 0.1541, -0.0446, -0.0685, -0.0274, -0.2238, -0.0505, -0.0708, -0.6247], [0.1403, -0.2584, 0.1304, -0.0171, -0.0685, 0.2828, -0.0405, 0.3866, -0.119, -0.4333], [0.0534, -0.0371, 0.1572, -0.0417, -0.2144, -0.0029, -0.2381, 0.0914, 0.0035, -0.8989], [0.1215, -0.2896, 0.2137, 0.0328, 0.0702, 0.0224, -0.2786, 0.3614, -0.1701, -0.0532], [-0.0422, -0.089, 0.0389, 0.1817, 0.0156, 0.145, -0.172, 0.3351, -0.0301, -0.4854], [-0.0714, -0.1694, 0.1079, -0.0154, -0.1311, -0.003, -0.2569, 0.1387, -0.112, -0.552], [-0.0748, -0.3086, 0.0798, 0.0623, -0.048, -0.1394, -0.2492, 0.3116, -0.2054, -0.4166], [0.0151, -0.2743, 0.0654, 0.0146, -0.0517, 0.0617, -0.1254, 0.1941, -0.0982, -0.2827], [0.1758, -0.3531, 0.1766, 0.1337, 0.0227, -0.0673, -0.4331, -0.0484, -0.2082, -0.4752], [0.1332, -0.2361, 0.182, 0.0505, -0.0122, -0.0509, -0.2213, 0.1685, -0.0362, -0.1515], [0.0872, -0.2519, 0.0529, -0.124, -0.0283, 0.0541, -0.0876, 0.1342, -0.2334, -0.5829], [0.1434, -0.3073, 0.0312, -0.1984, -0.2567, 0.0118, 0.0264, 0.1361, -0.0648, -0.8479], [0.133, -0.3083, 0.0281, 0.0106, -0.0258, -0.1042, -0.048, 0.1394, -0.0595, -0.4538], [0.0117, -0.0792, 0.1019, 0.0909, -0.0485, 0.1139, -0.0778, 0.0879, -0.0452, -0.5857], [0.0459, -0.0059, 0.1268, 0.0986, 0.0168, -0.2569, -0.3616, 0.2247, -0.041, -0.4115], [0.0527, -0.1985, -0.0047, -0.0341, -0.2311, 0.2574, -0.5896, 0.2249, -0.0717, -0.633], [0.1056, -0.2062, 0.1376, -0.1098, 0.2997, 0.0281, -0.1856, 0.1757, -0.021, -0.5138], [0.1447, -0.229, 0.1284, -0.1325, 0.1186, -0.2619, -0.2538, 0.0725, 0.0072, -1.0507], [-0.1122, -0.069, 0.1589, 0.1313, 0.0445, 0.0285, -0.2318, 0.139, -0.0754, -0.4923], [0.0825, 0.076, -0.0508, -0.1958, 0.0289, 0.067, -0.2372, 0.1146, -0.1605, -0.8109], [0.016, -0.2411, -0.0984, 0.0193, 0.0685, -0.0154, -0.4741, 0.145, 0.0788, -0.4628], [0.1414, -0.0247, 0.0802, 0.031, -0.0135, 0.2477, -0.3355, -0.1099, -0.078, -0.5183], [0.0783, -0.1491, -0.1434, -0.0636, -0.2592, -0.1233, -0.3365, 0.2516, -0.1969, -0.909], [0.1446, -0.3412, 0.0015, 0.0549, -0.1011, 0.0482, -0.4353, 0.1749, -0.1071, -0.0491], [0.0679, -0.0108, -0.063, 0.0313, 0.022, 0.1779, -0.3361, 0.3616, 0.1057, -0.39], [0.0014, -0.0202, 0.0513, 0.0715, 0.0735, 0.1609, -0.2323, -0.0581, -0.1662, -0.9525], [0.0727, -0.2555, 0.0333, 0.2598, 0.0116, 0.1378, -0.3753, 0.0679, -0.2233, -0.3584], [0.1504, -0.0833, 0.0073, -0.0879, -0.0413, 0.0165, -0.0888, 0.1345, -0.0918, -0.5684], [-0.0989, -0.1574, 0.1032, 0.055, -0.1942, 0.0273, 0.0908, 0.3317, 0.0941, -0.7467], [0.0713, -0.284, 0.1688, 0.0948, -0.0012, 0.0641, -0.1018, -0.0761, -0.1278, -0.3935], [0.1034, -0.0008, 0.1699, 0.1697, 0.2065, 0.2378, -0.3933, 0.025, 0.067, -0.7194], [0.0858, -0.039, 0.0252, 0.0066, 0.1109, 0.0665, -0.0066, -0.0257, -0.0138, -0.7897], [0.1399, -0.1216, -0.0296, -0.2803, 0.1379, 0.0975, 0.1562, 0.3358, -0.097, -0.8289], [0.1189, -0.1856, 0.1813, 0.1941, -0.1176, 0.1516, -0.493, 0.1101, -0.0305, -0.3248], [0.1033, -0.1557, 0.0673, 0.0659, -0.05, 0.0689, -0.4118, 0.2049, 0.0897, -0.0512], [0.0655, -0.1873, 0.1597, 0.0965, -0.1312, 0.0378, -0.413, 0.0335, 0.0784, -0.3266], [-0.194, -0.2508, 0.1919, 0.0684, -0.0436, -0.0021, -0.2253, 0.2453, 0.1666, -0.5004], [-0.1153, -0.2231, 0.1334, -0.0135, 0.0692, -0.1727, -0.3487, 0.249, 0.0527, -0.3619], [0.2373, -0.1852, 0.3377, 0.1608, -0.1439, 0.0452, -0.2209, 0.0262, -0.1227, -1.035], [0.0275, -0.1591, -0.0146, 0.1989, -0.0292, 0.0742, -0.1521, -0.1734, -0.04, -0.9281], [0.0663, 0.0265, 0.0997, 0.2044, -0.0902, 0.1633, -0.3674, 0.1873, 0.1273, -0.5506], [0.163, 0.1733, -0.0085, -0.0166, -0.0647, 0.0759, -0.2448, 0.2944, 0.0628, -0.5131], [0.2267, -0.2218, 0.3966, -0.0557, 0.1951, 0.0999, -0.1821, -0.024, -0.1184, -0.9269], [0.1038, 0.0207, 0.0445, 0.016, 0.0375, -0.0733, -0.1463, 0.0549, 0.0272, -0.6588], [0.1286, -0.1026, -0.1262, -0.1124, -0.1299, 0.0573, -0.6238, 0.1259, -0.0769, -1.066], [0.1785, -0.1751, -0.0434, 0.08, -0.1361, -0.3022, 0.0571, 0.286, -0.0138, -0.8322], [-0.0224, -0.2242, -0.0295, 0.1197, 0.0011, 0.1133, -0.0119, 0.02, -0.0098, -1.0395], [0.097, 0.0532, 0.0477, -0.0702, -0.004, 0.0771, -0.1989, -0.0568, 0.0244, -0.6899], [0.1835, -0.166, 0.0922, 0.0875, 0.0081, -0.2042, -0.0648, -0.0263, 0.1078, -0.9151], [0.0115, -0.3522, 0.1618, -0.0711, 0.0553, -0.038, -0.055, 0.2093, -0.1065, -0.4138], [-0.1678, -0.1693, 0.1565, -0.0172, 0.1806, 0.1139, -0.2891, 0.2305, -0.0777, -0.3405], [0.1803, -0.1846, 0.2212, -0.0702, 0.1304, 0.028, -0.1793, 0.3369, -0.1149, -0.1686], [0.1552, -0.2507, 0.1214, -0.0517, -0.0874, 0.1568, -0.0975, 0.2034, 0.0889, -0.3144], [0.1444, -0.1781, 0.1438, 0.1817, 0.0288, 0.034, -0.1058, 0.4046, -0.2183, -0.1982], [0.1179, -0.2092, 0.1701, 0.113, -0.0284, 0.0704, -0.1848, 0.1851, -0.0446, 0.0], [0.0848, -0.102, 0.053, 0.1001, 0.2153, -0.1013, -0.1182, 0.3213, -0.2275, -0.5544], [-0.0668, -0.0977, 0.058, 0.0777, 0.0266, -0.0248, -0.1126, 0.2475, -0.0714, -0.3396], [0.1455, -0.1721, 0.0925, 0.1048, -0.1617, 0.0134, 0.1881, 0.0896, -0.0703, -0.6458], [0.126, -0.0619, 0.2471, 0.0923, 0.159, 0.1996, -0.2086, 0.1424, -0.0371, -0.438], [0.2475, 0.1176, 0.1048, -0.1507, -0.0141, 0.0763, 0.0129, 0.1513, -0.0414, -0.9417], [-0.1598, -0.2982, 0.0543, 0.0299, 0.0348, 0.1723, -0.1856, -0.0496, -0.0758, -0.5093], [0.1415, -0.0673, -0.0252, -0.0118, 0.1751, 0.0702, -0.2925, 0.2145, -0.0691, -0.1908], [0.115, -0.2823, 0.0646, 0.0606, 0.0856, -0.014, -0.0045, 0.214, -0.1816, -0.2514], [0.1189, -0.2227, -0.0241, -0.0322, -0.035, -0.0266, -0.5637, 0.2393, -0.0497, -0.1241], [0.2034, -0.0096, -0.0904, -0.0751, -0.0061, -0.3214, -0.2935, 0.2794, 0.0597, -0.752], [0.1139, -0.2576, -0.022, -0.3431, 0.1545, 0.1139, -0.0681, 0.1239, 0.0313, -0.9227], [-0.106, -0.2441, -0.0338, -0.1383, -0.1478, 0.1243, -0.4305, 0.2266, -0.2433, -0.5437], [0.4014, -0.0747, 0.1168, 0.0498, -0.0315, -0.1142, -0.1131, 0.0363, -0.1119, -0.7057], [0.2265, -0.2308, 0.1009, 0.2029, -0.1601, -0.1254, -0.2225, 0.0636, -0.0651, -0.4951], [0.0035, -0.2437, 0.1432, 0.3047, 0.0655, -0.0039, -0.2498, -0.1196, -0.0985, -0.5507], [0.0873, -0.2658, 0.1597, -0.5172, 0.1319, 0.0426, 0.0137, 0.1311, -0.0974, -0.8097], [0.1326, -0.2559, -0.0071, 0.121, -0.057, -0.0019, -0.0012, 0.3533, -0.1426, -0.2307], [0.1449, -0.3811, -0.013, 0.1151, -0.0272, 0.1165, -0.1316, 0.0931, -0.0803, -0.2695], [0.1613, -0.2869, -0.0419, 0.044, -0.2945, 0.1163, -0.3266, -0.0499, -0.0552, -0.377], [0.0598, 0.0626, 0.0039, 0.0265, -0.1866, -0.0326, -0.3089, 0.165, -0.0671, -0.4851], [0.0162, -0.0832, 0.0509, 0.003, 0.0128, 0.0094, 0.0659, 0.2836, 0.0698, -0.5097], [0.1423, -0.4407, 0.12, -0.1473, 0.1442, 0.0582, -0.3553, 0.1455, -0.0626, -0.0946], [0.1747, -0.2489, 0.1205, 0.0661, -0.0268, 0.1966, -0.1708, 0.142, 0.1304, -0.2136], [0.2394, -0.0638, 0.1588, -0.2084, 0.0041, 0.1173, -0.4538, 0.3338, -0.0132, -0.1293], [0.0288, -0.2274, 0.0847, -0.1745, 0.0201, 0.025, -0.397, 0.2572, -0.1598, 0.0006], [0.0983, 0.2203, -0.1586, 0.1699, -0.139, 0.1328, -0.4066, 0.3893, -0.0673, -0.5762], [0.0239, -0.0077, -0.0658, -0.0992, -0.0877, -0.0237, -0.1528, 0.1105, -0.1774, -0.848], [0.1207, 0.0245, 0.0684, 0.0051, 0.2239, 0.0617, -0.1367, 0.1185, -0.1284, -1.181], [0.0738, -0.2681, -0.0422, 0.1016, 0.0358, -0.0834, -0.0551, 0.2286, -0.055, -0.2621], [0.0659, -0.0996, 0.0623, 0.1986, 0.0017, -0.0402, -0.3634, 0.4181, 0.12, -0.2351], [0.0468, -0.093, -0.0397, -0.19, 0.04, 0.0551, -0.11, 0.0724, -0.0294, -1.0525], [0.0567, -0.0089, 0.0251, 0.0513, 0.0238, -0.0006, -0.2187, 0.0311, 0.1057, -0.5225], [0.217, -0.0327, 0.1165, 0.0203, -0.115, 0.2063, -0.0659, 0.3229, -0.0405, -1.3699]]
plot_lists(tst_lsts, ref_lst, tst_lst, label1='Ref', label2='tst', title='Percent Change Diff', xlabel='Candlestick', ylabel='Percent Change')


In [41]:
import time

def format_unix_timestamp(unix_timestamp):
    # Convert the Unix timestamp to a time structure
    time_struct = time.localtime(unix_timestamp)
    
    # Format the time structure into the desired string format
    formatted_time = time.strftime("%d-%b-%Y %H:%M:%S", time_struct)
    
    return formatted_time

# Example usage:
unix_timestamp = 1609467300  # Example Unix timestamp
formatted_time = format_unix_timestamp(unix_timestamp)
print(formatted_time)  # Output: 13:Jul:2022 02:00:00 (example output)


01-Jan-2021 06:15:00


In [44]:
f_d = []
for i in aa:
    roi = i['avg_long_roi']
    tss = format_unix_timestamp(int(i['pattern_id'].split('_')[0]))
    avg_sms = i['avg_sm']
    pattern_occurrence = i['pattern_occurrence']
    f_d.append({'pattern_occurrence': pattern_occurrence, 'date_time': tss, 'long_roi': roi, 'avg_sm': avg_sms})

In [ ]:
for i in f_d:
    print(i)